# Exercício Prático: Classificação com Múltiplas Métricas (Capítulo 3)
Este notebook contém um exercício prático para aplicar os conceitos de classificação, avaliação de modelos e análise de erros abordados em aula.
Utilizaremos o dataset "Wine", um conjunto de dados público e pequeno com múltiplas classes.


In [2]:
# Execute esta célula para configurar o ambiente do notebook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score

# --- Carregando o Dataset ---
# O dataset "Wine" possui 178 amostras de vinhos,
# cada uma com 13 características químicas.
# O objetivo é classificar os vinhos em uma de 3 classes (cultivares).
wine = load_wine()
X = wine.data
y = wine.target

# Separação dos conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados para melhor desempenho do SGDClassifier
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Dataset 'Wine' carregado e pronto.")
print(f"Características do treino: {X_train_scaled.shape}")
print(f"Rótulos do treino: {y_train.shape}")

Dataset 'Wine' carregado e pronto.
Características do treino: (142, 13)
Rótulos do treino: (142,)


## Parte 1: Classificação Binária
Nesta seção, vamos adaptar o problema para uma tarefa de classificação binária, onde o objetivo será apenas identificar se um vinho pertence à classe 0 ou não.

### 1.1 - Adaptar o Dataset para 2 Classes
**Sua tarefa:** Crie as variáveis `y_train_0` e `y_test_0`. Elas devem conter `True` para as amostras que são da classe 0 e `False` para as outras.
Dica: Use uma comparação booleana com `y_train` e `y_test`.



In [3]:
# INSIRA SEU CÓDIGO AQUI
y_train_0 = (y_train == 0)

print("Primeiros 5 labels do y_train_0:", y_train_0[:5])

Primeiros 5 labels do y_train_0: [False False False False  True]


### 1.2 - Treinar um Classificador Binário e Fazer uma Predição
**Sua tarefa:**
1. Crie e treine uma instância do `SGDClassifier`. Use `random_state=42` para reprodutibilidade.
2. Use o classificador treinado para prever a classe da primeira instância do conjunto de treino (`X_train_scaled[0]`).
3. Imprima a classe prevista e a classe real.

In [7]:
sgd_clf = SGDClassifier(random_state=42)

# INSIRA SEU CÓDIGO AQUI
sgd_clf.fit(X_train_scaled, y_train_0)

predicao = sgd_clf.predict(X_train_scaled[0].reshape(1, -1))

print('Predito: ', predicao)
print('Real: ', y_train_0[0])

Predito:  [False]
Real:  False


## Parte 2: Avaliação de Desempenho (Binário)

### 2.1 - Medir Acurácia com Validação Cruzada
**Sua tarefa:** Use `cross_val_score` para avaliar seu `sgd_clf` com 3 folds (`cv=3`) e a métrica de acurácia. Imprima as pontuações de cada fold.

In [9]:
# INSIRA SEU CÓDIGO AQUI
acuracia_scores = cross_val_score(sgd_clf, X_train_scaled, y_train_0, cv=3, scoring="accuracy")

print("Acurácia em cada fold:", acuracia_scores)

Acurácia em cada fold: [0.95833333 1.         0.9787234 ]


### 2.2 - Baseline com `DummyClassifier`
**Sua tarefa:** Faça o mesmo que no passo anterior, mas usando um `DummyClassifier` com a estratégia "most_frequent". Isso nos dará uma baseline para comparar. O SGD é melhor que o baseline?



In [15]:
dummy_clf = DummyClassifier(strategy="most_frequent")

# INSIRA SEU CÓDIGO AQUI
dummy_clf.fit(X_train_scaled, y_train_0)
acuracia_dummy = cross_val_score(dummy_clf, X_train_scaled, y_train_0, cv = 3, scoring="accuracy")

print("Acurácia (Dummy) em cada fold:", acuracia_dummy)

Acurácia (Dummy) em cada fold: [0.6875     0.68085106 0.68085106]


### 2.3 - Matriz de Confusão, Precisão, Recall e F1-Score do SGDClassifier (binário)
**Sua tarefa:**
1. Obtenha as previsões para todo o conjunto de treino usando `cross_val_predict`.
2. Calcule e exiba a matriz de confusão.
3. Calcule e imprima a precisão, o recall e o F1-score.



In [ ]:
# INSIRA SEU CÓDIGO AQUI

crosPred = cross_val_predict(sgd_clf, X_train_scaled, y_train_0, cv = 3, method="predict")

print(confusion_matrix(crosPred,y_train_0))

print('Precisão: ',precision_score(crosPred,y_train_0))

print('recall: ', recall_score(crosPred,y_train_0))

print('F1-score: ', f1_score(crosPred,y_train_0))



[[95  1]
 [ 2 44]]
Precisão:  0.9777777777777777
recall:  0.9565217391304348
F1-score:  0.967032967032967


## Parte 3: Curvas de Avaliação (Binário)

### 3.1 - Obter Scores de Decisão
**Sua tarefa:** Use `cross_val_predict` novamente, mas desta vez para obter os *scores de decisão* (`decision_function`) em vez das previsões de classe.



In [47]:
# INSIRA SEU CÓDIGO AQUI
y_scores = cross_val_predict(sgd_clf, X_train_scaled, y_train_0, cv = 3, method='decision_function')

print("Scores de decisão (primeiros 5):", y_scores[:5])

Scores de decisão (primeiros 5): [ -57.48835356 -151.2779655   -70.67304845  -81.14164458   30.30061455]


Scores de decisão (primeiros 5): [ -57.48835356 -151.2779655   -70.67304845  -81.14164458   30.30061455]

### 3.2 - Plotar Curvas de Precisão-Recall e ROC
**Sua tarefa:**
1. Use a função `precision_recall_curve` para obter as precisões, recalls e limiares.
2. Plote a curva de Precisão vs. Recall.
3. Use a função `roc_curve` para obter a taxa de falsos positivos (fpr) e a taxa de verdadeiros positivos (tpr).
4. Plote a curva ROC.
5. Calcule e imprima a Área Sob a Curva ROC (AUC).



Área Sob a Curva ROC (AUC): 0.9975


In [49]:
# Curva Precisão-Recall

# INSIRA SEU CÓDIGO AQUI
precisions, recalls, thresholds_pr = precision_recall_curve(y_train_0, y_scores)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(recalls, precisions, "b-", linewidth=2)
plt.xlabel("Recall")
plt.ylabel("Precisão")
plt.title("Curva Precisão-Recall")
plt.grid(True)

# Curva ROC

# INSIRA SEU CÓDIGO AQUI
fpr, tpr, thresholds_roc = roc_curve(y_train_0, y_scores)

plt.subplot(1, 2, 2)
plt.plot(fpr, tpr, "b-", linewidth=2, label="SGD")
plt.plot([0, 1], [0, 1], 'k--') # Linha de referência
plt.xlabel("Taxa de Falsos Positivos")
plt.ylabel("Taxa de Verdadeiros Positivos (Recall)")
plt.title("Curva ROC")
plt.grid(True)
plt.show()

# Área Sob a Curva ROC (AUC)
roc_auc = roc_auc_score(y_train_0, y_scores)
print(f"Área Sob a Curva ROC (AUC): {roc_auc:.4f}")
# FIM DO SEU CÓDIGO

## Parte 4: Classificação Multiclasse e Análise de Erros

### 4.1 - Treinar o Classificador Multiclasse
**Sua tarefa:**
1. Treine uma nova instância do `SGDClassifier` (ou use a antiga), mas desta vez usando o `y_train` original, que contém todas as 3 classes.
2. Faça uma predição para a primeira instância do conjunto de treino e imprima a classe prevista e a real.



In [54]:
# INSIRA SEU CÓDIGO AQUI

sgd_clf_multi = SGDClassifier(random_state=42)
sgd_clf_multi.fit(X_train_scaled, y_train)

predict_2 = sgd_clf_multi.predict(X_train_scaled[0].reshape(1, -1))

print('Predito: ', predict_2)
print('Real: ', y_train[0])


Predito:  [2]
Real:  2


### 4.2 - Análise de Erros com Matriz de Confusão
**Sua tarefa:**
1. Use `cross_val_predict` para obter as previsões no conjunto de treino para o modelo multiclasse.
2. Plote a matriz de confusão normalizada (use `ConfusionMatrixDisplay.from_predictions` com `normalize='true'`).
3. Com base na matriz, identifique quais classes o modelo mais confunde.



In [67]:
y_train_pred_multi = cross_val_predict(sgd_clf_multi, X_train_scaled, y_train, cv=3)

# INSIRA SEU CÓDIGO AQUI
matriz = ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred_multi, normalize='true')

